# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --file establishments.json --jsonArray`

In [1]:
# Import dependencies
from pymongo import MongoClient
import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
establishments = db['establishments']
document = establishments.find_one()
pprint.pprint(document)

{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67d72572949f5057b0e8d869'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
} 

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('67d72b9e5d3ce8355dc07b67'), acknowledged=True)

In [10]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
results = establishments.find(query)

for result in results:
    pprint.pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67d72b9e5d3ce8355dc07b67'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}  # Project only BusinessTypeID and BusinessType

result = establishments.find_one(query, fields)

pprint.pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "_id": 0}
business_type_id_doc = establishments.find_one(query, fields)

if business_type_id_doc:
    business_type_id = business_type_id_doc["BusinessTypeID"]

    # Update the new restaurant
    establishments.update_one(
        {"BusinessName": "Penang Flavours"},
        {"$set": {"BusinessTypeID": business_type_id}}
    )
    print(f"Penang Flavours updated with BusinessTypeID: {business_type_id}")
else:
    print("BusinessTypeID not found for Restaurant/Cafe/Canteen")

Penang Flavours updated with BusinessTypeID: 1


In [14]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint.pprint(updated_restaurant)


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67d72b9e5d3ce8355dc07b67'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName 'Dover': {dover_count}")

Number of documents with LocalAuthorityName 'Dover': 994


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
dover_delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Number of documents deleted: {dover_delete_result.deleted_count}")

Number of documents deleted: 994


In [17]:
# Check if any remaining documents include Dover
dover_check = establishments.find_one({"LocalAuthorityName": "Dover"})
print(f"Remaining Dover document: {dover_check}")

Remaining Dover document: None


In [18]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()
pprint.pprint(remaining_document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67d72572949f5057b0e8db50'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [
    {'$set': {
        "geocode.longitude": {'$toDecimal': "$geocode.longitude"},
        "geocode.latitude": {'$toDecimal': "$geocode.latitude"}
    }}
])

print("Longitude and latitude converted to Decimal.")

Longitude and latitude converted to Decimal.


Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [
    {'$set': {
        "RatingValue": {'$toInt': "$RatingValue"}
    }}
])

print("RatingValue converted to Integer.")

RatingValue converted to Integer.


In [22]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one()

if sample_document and sample_document.get("geocode") and sample_document["geocode"].get("longitude") and sample_document["geocode"].get("latitude") and sample_document.get("RatingValue"):
    print(f"Longitude type: {type(sample_document['geocode']['longitude'])}")
    print(f"Latitude type: {type(sample_document['geocode']['latitude'])}")
    print(f"RatingValue type: {type(sample_document['RatingValue'])}")
else:
    print("Could not retrieve sample document or document does not contain required fields.")

Longitude type: <class 'bson.decimal128.Decimal128'>
Latitude type: <class 'bson.decimal128.Decimal128'>
RatingValue type: <class 'int'>
